# Visualization of Coincident ICESat-2, Sentinel, and Argo Float Data

ICESat-2 hackweek final project 
June 16, 2020  
Nicole Abib

In [1]:
import os
from icepyx import icesat2data as ipd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# from shapely.geometry import Point, Polygon
import warnings
warnings.filterwarnings('ignore')
# magic function to enable interactive plotting
%matplotlib widget 

In [2]:
# Define path to sample data
argo_fn = '/home/jovyan/shared/leading_to_phytoplankton/floats_highlats.csv'
# is2_fn = 
# s2_fn = 

In [3]:
# Quick check of csv file contents
!head $argo_fn

-64.977,-155.98,2018,1,2,8,57,6.84,0.0041413,-0.0327,34.033,2.4624
-64.977,-155.98,2018,1,2,8,57,10.58,0.0043545,-0.0316,34.033,2.5056
-64.977,-155.98,2018,1,2,8,57,14.94,0.0041301,-0.0307,34.032,2.8152
-64.977,-155.98,2018,1,2,8,57,20.97,0.0042085,-0.0785,34.036,2.628
-64.977,-155.98,2018,1,2,8,57,26.04,0.0045563,-0.1061,34.037,3.0096
-64.977,-155.98,2018,1,2,8,57,31.18,0.0047467,-0.2544,34.042,3.5136
-64.977,-155.98,2018,1,2,8,57,35.67,0.0033291,-1.6271,34.186,3.1104
-64.977,-155.98,2018,1,2,8,57,40.21,0.0029811,-1.703,34.196,3.5064
-64.977,-155.98,2018,1,2,8,57,45.24,0.0036319,-1.6948,34.2,3.708
-64.977,-155.98,2018,1,2,8,57,50.29,0.0020383,-1.7466,34.206,2.412


In [4]:
# Load the csv file with Pandas
argo_df = pd.read_csv(argo_fn)
# Add column names defined in the metadata
argo_df.columns = ['lat', 'lon', 'year', 'month', 'day', 'hour', 
                   'minute', 'Depth (m)', 'bbp (700 nm)', 'temperature', 
                   'salinity', 'chlorophyll']
# Inspect the dataframe
argo_df

,lat,lon,year,month,day,hour,minute,Depth (m),bbp (700 nm),temperature,salinity,chlorophyll
0,-64.977,-155.980,2018,1.0,2.0,8.0,57.0,10.58,0.004354,-0.0316,34.033,2.50560
1,-64.977,-155.980,2018,1.0,2.0,8.0,57.0,14.94,0.004130,-0.0307,34.032,2.81520
2,-64.977,-155.980,2018,1.0,2.0,8.0,57.0,20.97,0.004209,-0.0785,34.036,2.62800
3,-64.977,-155.980,2018,1.0,2.0,8.0,57.0,26.04,0.004556,-0.1061,34.037,3.00960
4,-64.977,-155.980,2018,1.0,2.0,8.0,57.0,31.18,0.004747,-0.2544,34.042,3.51360
...,...,...,...,...,...,...,...,...,...,...,...,...
449725,-50.810,4.359,2019,1.0,2.0,0.0,34.0,134.00,0.002312,2.3460,33.906,0.85765
449726,-50.810,4.359,2019,1.0,2.0,0.0,34.0,136.00,0.001488,2.2490,33.919,0.52873
449727,-50.810,4.359,2019,1.0,2.0,0.0,34.0,138.00,0.001300,2.1780,33.929,0.37503
449728,-50.810,4.359,2019,1.0,2.0,0.0,34.0,140.00,0.001373,2.1080,33.943,0.27205


In [5]:
# Check data types
argo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 449730 entries, 0 to 449729
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   lat           449730 non-null  float64
 1   lon           449730 non-null  float64
 2   year          449730 non-null  int64  
 3   month         449729 non-null  float64
 4   day           449729 non-null  float64
 5   hour          449729 non-null  float64
 6   minute        449729 non-null  float64
 7   Depth (m)     449729 non-null  float64
 8   bbp (700 nm)  449729 non-null  float64
 9   temperature   449729 non-null  float64
 10  salinity      449729 non-null  float64
 11  chlorophyll   449729 non-null  float64
dtypes: float64(11), int64(1)
memory usage: 41.2 MB


In [6]:
# Check that column labels are correct
argo_df.columns

Index(['lat', 'lon', 'year', 'month', 'day', 'hour', 'minute', 'Depth (m)',
       'bbp (700 nm)', 'temperature', 'salinity', 'chlorophyll'],
      dtype='object')

In [7]:
# Check head
argo_df.head()

,lat,lon,year,month,day,hour,minute,Depth (m),bbp (700 nm),temperature,salinity,chlorophyll
0,-64.977,-155.98,2018,1.0,2.0,8.0,57.0,10.58,0.004354,-0.0316,34.033,2.5056
1,-64.977,-155.98,2018,1.0,2.0,8.0,57.0,14.94,0.004130,-0.0307,34.032,2.8152
2,-64.977,-155.98,2018,1.0,2.0,8.0,57.0,20.97,0.004209,-0.0785,34.036,2.6280
3,-64.977,-155.98,2018,1.0,2.0,8.0,57.0,26.04,0.004556,-0.1061,34.037,3.0096
4,-64.977,-155.98,2018,1.0,2.0,8.0,57.0,31.18,0.004747,-0.2544,34.042,3.5136


In [8]:
# Check tail
argo_df.tail()

,lat,lon,year,month,day,hour,minute,Depth (m),bbp (700 nm),temperature,salinity,chlorophyll
449725,-50.81,4.359,2019,1.0,2.0,0.0,34.0,134.0,0.002312,2.346,33.906,0.85765
449726,-50.81,4.359,2019,1.0,2.0,0.0,34.0,136.0,0.001488,2.249,33.919,0.52873
449727,-50.81,4.359,2019,1.0,2.0,0.0,34.0,138.0,0.001300,2.178,33.929,0.37503
449728,-50.81,4.359,2019,1.0,2.0,0.0,34.0,140.0,0.001373,2.108,33.943,0.27205
449729,-50.81,4.359,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
argo_df_nan = argo_df.replace(99999, np.nan)

In [10]:
# Create a scatter plot showing data locations
ax = argo_df_nan.plot(x='lon', y='lat', kind='scatter', c='k', s=1)
ax.set_title('Location of Argo Floats')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Location of Argo Floats')

In [11]:
import geopandas as gpd
argo_gdf = gpd.GeoDataFrame(argo_df_nan,geometry = gpd.points_from_xy(argo_df_nan['lon'],
                                                                     argo_df_nan['lat']),
                                                                     crs='EPSG:4326')

In [12]:
argo_gdf.head()

,lat,lon,year,month,day,hour,minute,Depth (m),bbp (700 nm),temperature,salinity,chlorophyll,geometry
0,-64.977,-155.98,2018,1.0,2.0,8.0,57.0,10.58,0.004354,-0.0316,34.033,2.5056,POINT (-155.98000 -64.97700)
1,-64.977,-155.98,2018,1.0,2.0,8.0,57.0,14.94,0.004130,-0.0307,34.032,2.8152,POINT (-155.98000 -64.97700)
2,-64.977,-155.98,2018,1.0,2.0,8.0,57.0,20.97,0.004209,-0.0785,34.036,2.6280,POINT (-155.98000 -64.97700)
3,-64.977,-155.98,2018,1.0,2.0,8.0,57.0,26.04,0.004556,-0.1061,34.037,3.0096,POINT (-155.98000 -64.97700)
4,-64.977,-155.98,2018,1.0,2.0,8.0,57.0,31.18,0.004747,-0.2544,34.042,3.5136,POINT (-155.98000 -64.97700)


In [13]:
argo_gdf.plot('temperature', markersize=1, cmap='inferno', legend=True);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
countries_url = 'https://d2ad6b4ur7yvpq.cloudfront.net/naturalearth-3.3.0/ne_110m_land.geojson'

In [15]:
countries_gdf = gpd.read_file(countries_url)

In [16]:
countries_gdf.head()

,scalerank,featureclass,geometry
0,1,Country,"POLYGON ((-59.57209 -80.04018, -59.86585 -80.5..."
1,1,Country,"POLYGON ((-159.20818 -79.49706, -161.12760 -79..."
2,1,Country,"POLYGON ((-45.15476 -78.04707, -43.92083 -78.4..."
3,1,Country,"POLYGON ((-121.21151 -73.50099, -119.91885 -73..."
4,1,Country,"POLYGON ((-125.55957 -73.48135, -124.03188 -73..."


In [18]:
caxis_max = argo_df.chlorophyll.mean() + argo_df.chlorophyll.std()
caxis_min = argo_df.chlorophyll.mean() - argo_df.chlorophyll.std()
ax = argo_gdf.plot('chlorophyll', markersize=1, cmap='inferno', legend=True, vmin=0, vmax = 0.1,legend_kwds={'label': "Chlorophyll",'orientation': "horizontal"});
countries_gdf.plot(ax=ax, facecolor='none', edgecolor='0.5', linewidth=0.5);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …